In [1]:
# Install NVIDIA drivers
!git clone https://github.com/NVIDIA/apex
% cd apex
!pip install -v --disable-pip-version-check --no-cache-dir ./
% cd ..

Cloning into 'apex'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 7872 (delta 10), reused 7 (delta 0), pack-reused 7835
Receiving objects: 100% (7872/7872), 13.99 MiB | 27.04 MiB/s, done.
Resolving deltas: 100% (5357/5357), done.
/content/apex
Created temporary directory: /tmp/pip-ephem-wheel-cache-jbwzbdb2
Created temporary directory: /tmp/pip-req-tracker-zkl6q424
Created requirements tracker '/tmp/pip-req-tracker-zkl6q424'
Created temporary directory: /tmp/pip-install-7msi1opx
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-cji__zbi
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-zkl6q424'
    Running setup.py (path:/tmp/pip-req-build-cji__zbi/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.7.0+cu101


    running egg_info
    creating /tmp/pip-req-buil

In [2]:
!pip install datasets transformers -q

     |████████████████████████████████| 163kB 14.8MB/s 
     |████████████████████████████████| 1.5MB 27.0MB/s 
     |████████████████████████████████| 245kB 49.6MB/s 
     |████████████████████████████████| 17.7MB 212kB/s 
     |████████████████████████████████| 890kB 44.6MB/s 
     |████████████████████████████████| 2.9MB 50.7MB/s 


In [4]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 57442 (delta 19), reused 11 (delta 1), pack-reused 57380
Receiving objects: 100% (57442/57442), 42.86 MiB | 27.14 MiB/s, done.
Resolving deltas: 100% (40282/40282), done.


In [3]:
import numpy as np
import torch
import datasets
import transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline
from torch.utils.data import Dataset, DataLoader

## Loading the corpora

In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
train_path = "/content/gdrive/MyDrive/train.txt"
eval_path = "/content/gdrive/MyDrive/eval.txt"

In [ ]:
model_dir = 'model'
!mkdir {model_dir}

## Fine-tuning

In [32]:
cmd ='''python /content/transformers/examples/language-modeling/run_mlm.py \
    --mlm 0.15 \
    --model_name_or_path bert-base-uncased \
    --train_file {} \
    --validation_file {} \
    --do_train \
    --do_eval \
    --output_dir {} \
    --fp16 \
    --overwrite_output_dir \
    --num_train_epochs 1 \
    --max_seq_length 128 \
    --save_total_limit 2 \
    --save_steps 40000 \
    '''.format(train_path, eval_path, model_dir)

In [33]:
! {cmd}

2020-12-24 11:01:04.105206: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/24/2020 11:01:05 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
12/24/2020 11:01:05 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='model', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec24_11-01-05_eb2c441f5812', logging_first_step=False, logging_

## Loading the fine-tuned model and examining its performance on the MLM task

In [53]:
trained_model = transformers.BertForMaskedLM.from_pretrained(model_dir)
trained_tokenizer = transformers.BertTokenizer.from_pretrained(model_dir)

In [56]:
fill_mask = pipeline(
    "fill-mask",
    model= trained_model,
    tokenizer=trained_tokenizer
)

In [227]:
fill_mask("The capital of France is [MASK].")

[{'score': 0.35380667448043823,
  'sequence': '[CLS] the capital of france is paris. [SEP]',
  'token': 3000,
  'token_str': 'paris'},
 {'score': 0.23379506170749664,
  'sequence': '[CLS] the capital of france is strasbourg. [SEP]',
  'token': 18104,
  'token_str': 'strasbourg'},
 {'score': 0.08326232433319092,
  'sequence': '[CLS] the capital of france is brussels. [SEP]',
  'token': 9371,
  'token_str': 'brussels'},
 {'score': 0.06250171363353729,
  'sequence': '[CLS] the capital of france is luxembourg. [SEP]',
  'token': 10765,
  'token_str': 'luxembourg'},
 {'score': 0.03878603130578995,
  'sequence': '[CLS] the capital of france is nice. [SEP]',
  'token': 3835,
  'token_str': 'nice'}]

## Uploading the model to hugginface

In [200]:
!transformers-cli login

2020-12-24 15:12:00.315576: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: lukabor
Password: 
Login successful
Your token: owGAinodNDYLgcfcUpolBRHTXjXREYUTuMKUMolKgeWJumtyzPUJnIGyrlUhPwGbDLXNdPpRsGmcwKAwOQrupEiMeLpTHvOQEQAPhensMTRhKnfudyWiPetvUcaXtnBN 

Your token has been saved to /root/.huggingface/token


In [201]:
!transformers-cli repo create europarl-mlm

2020-12-24 15:12:22.205974: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
git version 2.17.1
Error: unknown flag: --version

Sorry, no usage text found for "git-lfs"

You are about to create lukabor/europarl-mlm
Proceed? [Y/n] Y

Your repo now lives at:
  https://huggingface.co/lukabor/europarl-mlm

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/lukabor/europarl-mlm



In [202]:
!git clone https://huggingface.co/lukabor/europarl-mlm

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'europarl-mlm'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0)
Unpacking objects: 100% (3/3), done.


In [203]:
%cd europarl-mlm

/content/europarl-mlm


In [204]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [221]:
!cp /content/model/config.json /content/europarl-mlm/
!cp /content/model/pytorch_model.bin /content/europarl-mlm/
!cp /content/model/special_tokens_map.json /content/europarl-mlm/
!cp /content/model/tokenizer_config.json /content/europarl-mlm/
!cp /content/model/train_results.txt /content/europarl-mlm/
!cp /content/model/training_args.bin /content/europarl-mlm/
!cp /content/model/vocab.txt /content/europarl-mlm/

In [213]:
!git lfs track .

Tracking "."
Pattern . matches forbidden file .gitattributes. If you would like to track .gitattributes, modify .gitattributes manually.


In [214]:
!git add .gitattributes

In [222]:
!git add .

In [223]:
!git commit -m 'uloading the model'

[main 137e53f] uloading the model
 6 files changed, 30530 insertions(+)
 create mode 100644 pytorch_model.bin
 create mode 100644 special_tokens_map.json
 create mode 100644 tokenizer_config.json
 create mode 100644 train_results.txt
 create mode 100644 training_args.bin
 create mode 100644 vocab.txt


Git LFS: (2 of 2 files) 417.85 MB / 417.85 MB
Counting objects: 8, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (7/7), done.
Writing objects: 100% (8/8), 109.05 KiB | 5.19 MiB/s, done.
Total 8 (delta 0), reused 0 (delta 0)
To https://huggingface.co/lukabor/europarl-mlm
   d732aa4..137e53f  main -> main
